In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 233 kB in 1s (178 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)



In [4]:
# 2. Create a temporary view of the DataFrame.
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
#2.5 Display DataFrame schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# Import functions
from pyspark.sql.functions import col, year, avg, round

# Convert 'price' column to a numeric type
df = df.withColumn("price", df["price"].cast("float"))

# Extract the year from the 'date' column
df = df.withColumn("year", year(df["date"]))

# Filter for four-bedroom houses
four_bedroom_df = df.filter(col("bedrooms") == "4")

# Calculate the average price for each year
avg_price_for_four_bdrm_per_year = four_bedroom_df.groupBy("year").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_for_four_bdrm_per_year = avg_price_for_four_bdrm_per_year.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_for_four_bdrm_per_year.show()


+----+------------------+-----------------+
|year|         avg_price|avg_price_rounded|
+----+------------------+-----------------+
|2022| 296363.8845050215|        296363.88|
|2019| 300263.6955128205|         300263.7|
|2020|298353.78003169573|        298353.78|
|2021|   301819.44398864|        301819.44|
+----+------------------+-----------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Convert 'price' column to a numeric type
df = df.withColumn("price", df["price"].cast("float"))

# Extract the year from the 'date_built' column
df = df.withColumn("year_built", year(df["date_built"]))

# Filter for homes with 3 bedrooms and 3 bathrooms
three_bed_three_bath_df = df.filter((col("bedrooms") == "3") & (col("bathrooms") == "3"))

# Calculate the average price for each year built
avg_price_by_year_built = three_bed_three_bath_df.groupBy("year_built").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_year_built = avg_price_by_year_built.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_year_built.show()


+----------+------------------+-----------------+
|year_built|         avg_price|avg_price_rounded|
+----------+------------------+-----------------+
|      2015| 288770.2966101695|         288770.3|
|      2013|295962.27145085804|        295962.27|
|      2014| 290852.2661870504|        290852.27|
|      2012| 293683.1872074883|        293683.19|
|      2016|  290555.073964497|        290555.07|
|      2010|  292859.615942029|        292859.62|
|      2011|291117.46706586826|        291117.47|
|      2017| 292676.7887740029|        292676.79|
+----------+------------------+-----------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Convert 'price' and 'sqft_living' columns to numeric types
df = df.withColumn("price", df["price"].cast("float"))
df = df.withColumn("sqft_living", df["sqft_living"].cast("float"))

# Extract the year from the 'date_built' column
df = df.withColumn("year_built", year(df["date_built"]))

# Filter for homes with specific criteria
filtered_df = df.filter(
    (col("bedrooms") == "3") &
    (col("bathrooms") == "3") &
    (col("floors") == "2") &
    (col("sqft_living") >= 2000)
)

# Calculate the average price for each year built
avg_price_by_year_built = filtered_df.groupBy("year_built").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_year_built = avg_price_by_year_built.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_year_built.show()

+----------+------------------+-----------------+
|year_built|         avg_price|avg_price_rounded|
+----------+------------------+-----------------+
|      2015| 297609.9679144385|        297609.97|
|      2013|      303676.79375|        303676.79|
|      2014| 298264.7183908046|        298264.72|
|      2012|307539.97402597405|        307539.97|
|      2016| 293965.1046511628|         293965.1|
|      2010| 285010.2215909091|        285010.22|
|      2011| 276553.8128654971|        276553.81|
|      2017|280317.57692307694|        280317.58|
+----------+------------------+-----------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Convert 'price' column to a numeric type
df = df.withColumn("price", df["price"].cast("float"))

# Filter for homes with a price greater than or equal to $350,000
filtered_df = df.filter(col("price") >= 350000)

# Calculate the average price for each "view" rating
avg_price_by_view = filtered_df.groupBy("view").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_view = avg_price_by_view.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_view.show()

# Measure the runtime
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+-----------------+
|view|         avg_price|avg_price_rounded|
+----+------------------+-----------------+
|   7| 403005.7709497207|        403005.77|
|  51| 788128.2083333334|        788128.21|
|  15|404673.29545454547|         404673.3|
|  54| 798684.8235294118|        798684.82|
|  11|399548.11891891895|        399548.12|
|  29| 397771.6512820513|        397771.65|
|  69| 750537.9411764706|        750537.94|
|  42|396964.50241545896|         396964.5|
|  87|         1072285.2|        1072285.2|
|  73| 752861.1764705882|        752861.18|
|  64| 767036.6666666666|        767036.67|
|   3| 398867.5964912281|         398867.6|
|  30|          397862.0|         397862.0|
|  34| 401419.7451923077|        401419.75|
|  59|          791453.0|         791453.0|
|   8|398592.70658682636|        398592.71|
|  28|402124.62176165805|        402124.62|
|  22| 402022.6847826087|        402022.68|
|  85|1056336.7435897435|       1056336.74|
|  16| 399586.5311004785|       

In [10]:
# 7. Cache the the temporary table home_sales.

# Convert 'price' column to a numeric type
df = df.withColumn("price", df["price"].cast("float"))

# Create a temporary SQL view for the DataFrame
df.createOrReplaceTempView("home_sales")

# Cache the DataFrame
df.cache()

# Filter for homes with a price greater than or equal to $350,000
filtered_df = df.filter(col("price") >= 350000)

# Calculate the average price for each "view" rating
avg_price_by_view = filtered_df.groupBy("view").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_view = avg_price_by_view.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_view.show()

# Measure the runtime
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+-----------------+
|view|         avg_price|avg_price_rounded|
+----+------------------+-----------------+
|   7| 403005.7709497207|        403005.77|
|  51| 788128.2083333334|        788128.21|
|  15|404673.29545454547|         404673.3|
|  54| 798684.8235294118|        798684.82|
|  11|399548.11891891895|        399548.12|
|  29| 397771.6512820513|        397771.65|
|  69| 750537.9411764706|        750537.94|
|  42|396964.50241545896|         396964.5|
|  87|         1072285.2|        1072285.2|
|  73| 752861.1764705882|        752861.18|
|  64| 767036.6666666666|        767036.67|
|   3| 398867.5964912281|         398867.6|
|  30|          397862.0|         397862.0|
|  34| 401419.7451923077|        401419.75|
|  59|          791453.0|         791453.0|
|   8|398592.70658682636|        398592.71|
|  28|402124.62176165805|        402124.62|
|  22| 402022.6847826087|        402022.68|
|  85|1056336.7435897435|       1056336.74|
|  16| 399586.5311004785|       

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Convert 'price' column to a numeric type
df = df.withColumn("price", df["price"].cast("float"))

# Create a temporary SQL view for the DataFrame
df.createOrReplaceTempView("home_sales")

# Uncached Query
start_time_uncached = time.time()

# Filter for homes with a price greater than or equal to $350,000
filtered_df_uncached = df.filter(col("price") >= 350000)

# Calculate the average price for each "view" rating
avg_price_by_view_uncached = filtered_df_uncached.groupBy("view").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_view_uncached = avg_price_by_view_uncached.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_view_uncached.show()

uncached_runtime = time.time() - start_time_uncached

# Cache the DataFrame
df.cache()

# Cached Query
start_time_cached = time.time()

# Filter for homes with a price greater than or equal to $350,000 using the cached data
filtered_df_cached = df.filter(col("price") >= 350000)

# Calculate the average price for each "view" rating
avg_price_by_view_cached = filtered_df_cached.groupBy("view").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_view_cached = avg_price_by_view_cached.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_view_cached.show()

cached_runtime = time.time() - start_time_cached

# Print runtimes
print(f"Uncached Runtime: {uncached_runtime} seconds")
print(f"Cached Runtime: {cached_runtime} seconds")


+----+------------------+-----------------+
|view|         avg_price|avg_price_rounded|
+----+------------------+-----------------+
|   7| 403005.7709497207|        403005.77|
|  51| 788128.2083333334|        788128.21|
|  15|404673.29545454547|         404673.3|
|  54| 798684.8235294118|        798684.82|
|  11|399548.11891891895|        399548.12|
|  29| 397771.6512820513|        397771.65|
|  69| 750537.9411764706|        750537.94|
|  42|396964.50241545896|         396964.5|
|  87|         1072285.2|        1072285.2|
|  73| 752861.1764705882|        752861.18|
|  64| 767036.6666666666|        767036.67|
|   3| 398867.5964912281|         398867.6|
|  30|          397862.0|         397862.0|
|  34| 401419.7451923077|        401419.75|
|  59|          791453.0|         791453.0|
|   8|398592.70658682636|        398592.71|
|  28|402124.62176165805|        402124.62|
|  22| 402022.6847826087|        402022.68|
|  85|1056336.7435897435|       1056336.74|
|  16| 399586.5311004785|       

In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Convert 'price' column to a numeric type
df = df.withColumn("price", df["price"].cast("float"))

# Write the DataFrame partitioned by the "date_built" field
df.write.parquet("parquet-date_built", mode="overwrite")


In [14]:
# 11. Read the parquet formatted data.

p_df = spark.read.parquet("parquet-date_built")


In [15]:
# 12. Create a temporary table for the parquet data.

p_df.createOrReplaceTempView('p_descriptions')

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Replace "parquet-date_built" with the actual path
parquet_path = "/content/parquet-date_built"

# Read Parquet data into a DataFrame
p_df = spark.read.parquet(parquet_path)

# Create a temporary SQL view for the DataFrame
p_df.createOrReplaceTempView('p_descriptions')

# Uncached Query
start_time_uncached = time.time()

# Filter for homes with a price greater than or equal to $350,000
filtered_p_df_uncached = p_df.filter(col("price") >= 350000)

# Calculate the average price for each "view" rating
avg_price_by_view_uncached = filtered_p_df_uncached.groupBy("view").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_view_uncached = avg_price_by_view_uncached.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_view_uncached.show()

uncached_runtime = time.time() - start_time_uncached

# Cached Query
start_time_cached = time.time()

# Filter for homes with a price greater than or equal to $350,000 using the temporary table
filtered_p_temp_table = spark.sql("SELECT * FROM p_descriptions WHERE price >= 350000")

# Calculate the average price for each "view" rating
avg_price_by_view_temp_table = filtered_p_temp_table.groupBy("view").agg(avg("price").alias("avg_price"))

# Round the average price to two decimal places
avg_price_by_view_temp_table = avg_price_by_view_temp_table.withColumn("avg_price_rounded", round("avg_price", 2))

# Show the result
avg_price_by_view_temp_table.show()

temp_table_runtime = time.time() - start_time_cached

# Print runtimes
print(f"Uncached Runtime: {uncached_runtime} seconds")
print(f"Temp Table Runtime: {temp_table_runtime} seconds")


+----+------------------+-----------------+
|view|         avg_price|avg_price_rounded|
+----+------------------+-----------------+
|   7| 403005.7709497207|        403005.77|
|  51| 788128.2083333334|        788128.21|
|  15|404673.29545454547|         404673.3|
|  54| 798684.8235294118|        798684.82|
|  11|399548.11891891895|        399548.12|
|  29| 397771.6512820513|        397771.65|
|  69| 750537.9411764706|        750537.94|
|  42|396964.50241545896|         396964.5|
|  87|         1072285.2|        1072285.2|
|  73| 752861.1764705882|        752861.18|
|  64| 767036.6666666666|        767036.67|
|   3| 398867.5964912281|         398867.6|
|  30|          397862.0|         397862.0|
|  34| 401419.7451923077|        401419.75|
|  59|          791453.0|         791453.0|
|   8|398592.70658682636|        398592.71|
|  28|402124.62176165805|        402124.62|
|  22| 402022.6847826087|        402022.68|
|  85|1056336.7435897435|       1056336.74|
|  16| 399586.5311004785|       

In [17]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False